In [5]:
import pandas as pd;
import datetime as dt;
import numpy as np;
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import scale
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [6]:
training = pd.read_csv('training2.csv')
del training['time']


X = training.iloc[:, :-1]
y = training.iloc[:, -1:]

X_max = X.max()
X = X / X_max

#NN

hl = [40, 30, 20]
reg = MLPRegressor(\
                   solver='adam',\
                   hidden_layer_sizes = hl,\
                   max_iter = 200000,\
                   learning_rate = 'adaptive',\
                   learning_rate_init = 0.1,\
                   activation = 'relu') 
reg.fit(X, y)

# Xgb
xgb_model = xgb.XGBRegressor(max_depth=5, n_estimators=50, min_child_weight=1, gamma=0.9).fit(X,y)

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1266: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
submission = pd.read_csv('submission2.csv')
submission_ori = submission.copy()
del submission['time_window']
del submission['volume']

X_submission = submission[X.columns] / X_max
submission_ori['volume'] = xgb_model.predict(X_submission) / 3
submission['volume'] = submission_ori['volume'] * 3
submission_ori = submission_ori[submission_ori.columns[0:4]]
submission_ori.to_csv('submission2_result.csv', index=False)